# Import and Download required Libraries and files

In [10]:
!git clone https://github.com/aniketdhole07/Know-Your-Music
!pip install pydub
from os import path
from pydub import AudioSegment
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
import librosa

Cloning into 'Know-Your-Music'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 6 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (6/6), done.


# Data Cleaning and Model Creation

In [11]:
#Data Cleaning
df = pd.read_csv('/content/Know-Your-Music/data.csv')
df = df.drop(['beats'], axis=1)
df['class_name'] = df['class_name'].astype('category')
df['class_label'] = df['class_name'].cat.codes
lookup_genre_name = dict(zip(df.class_label.unique(), df.class_name.unique()))
cols = list(df.columns)
cols.remove('label')
cols.remove('class_label')
cols.remove('class_name')

#Process Training Dara
X = df.iloc[:,1:28]
y = df['class_label']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=3)
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

#Create K-Nearest Neighbour Model
knn = KNeighborsClassifier(n_neighbors = 13)
knn.fit(X_train_scaled, y_train)
knn.score(X_test_scaled, y_test)

0.624

# Upload MP3 file

In [12]:
from google.colab import files
print("Upload mp3 file")
files.upload()

import glob, os
os.chdir("/content")
for file in glob.glob("*.mp3"):
  src=file
print(src)  

Upload mp3 file


Alan Walker On My Way(Mp3meet.com).mp3


# Fetching File Metadata For Prediction

In [9]:
dst = "op.wav"                                                       
sound = AudioSegment.from_mp3(src)
sound.export(dst, format="wav")

y, sr = librosa.load("op.wav")
onset_env = librosa.onset.onset_strength(y, sr)
tempo = librosa.beat.tempo(onset_envelope=onset_env, sr=sr)
y_harmonic, y_percussive = librosa.effects.hpss(y)
tempo, beat_frames = librosa.beat.beat_track(y=y_percussive,sr=sr)
chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
rmse = librosa.feature.rmse(y=y)
spec_centroid = librosa.feature.spectral_centroid(y, sr=sr)[0]
spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
spec_rolloff = librosa.feature.spectral_rolloff(y+0.01, sr=sr)[0]
zero_crossing = librosa.feature.zero_crossing_rate(y)
mfcc = librosa.feature.mfcc(y=y, sr=sr)
metadata_dict = {'tempo':tempo,'chroma_stft':np.mean(chroma_stft),'rmse':np.mean(rmse),
                  'spectral_centroid':np.mean(spec_centroid),'spectral_bandwidth':np.mean(spec_bw), 
                  'rolloff':np.mean(spec_rolloff), 'zero_crossing_rates':np.mean(zero_crossing)}
for i in range(1,21):
    metadata_dict.update({'mfcc'+str(i):np.mean(mfcc[i-1])})

op=list(metadata_dict.values())
d1 =np.array(op)
data1 = scaler.transform([d1])
genre_prediction = knn.predict(data1)
print("----------------------------------------")
print(" Music Genre : ",end=" ")
print(lookup_genre_name[genre_prediction[0]])
print("----------------------------------------")

/usr/local/lib/python3.6/dist-packages/librosa/beat.py:306: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  hop_length=hop_length))
/usr/local/lib/python3.6/dist-packages/librosa/beat.py:306: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  hop_length=hop_length))


----------------------------------------
 Music Genre :  hiphop
----------------------------------------
